# First_stage
image to vector

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchsummary import summary
import matplotlib.pyplot as plt
import time
import os
import copy
import random
import PIL.ImageOps
from PIL import ImageMath,Image
from torch.utils.data import Dataset, DataLoader


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load('base.pt')
model.eval()
# model.freeze()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:604: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)


RecursiveScriptModule(
  original_name=Sequential
  (0): RecursiveScriptModule(
    original_name=Sequential
    (conv1): RecursiveScriptModule(original_name=Conv2d)
    (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
    (relu): RecursiveScriptModule(original_name=ReLU)
    (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
    (layer1): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=BasicBlock
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
        (relu): RecursiveScriptModule(original_name=ReLU)
        (conv2): RecursiveScriptModule(original_name=Conv2d)
        (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      )
      (1): RecursiveScriptModule(
        original_name=BasicBlock
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
        (relu):

In [3]:
# summary(model, (3, 224, 224))

In [4]:
# print(model)

In [5]:
model.to(device)

RecursiveScriptModule(
  original_name=Sequential
  (0): RecursiveScriptModule(
    original_name=Sequential
    (conv1): RecursiveScriptModule(original_name=Conv2d)
    (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
    (relu): RecursiveScriptModule(original_name=ReLU)
    (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
    (layer1): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=BasicBlock
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
        (relu): RecursiveScriptModule(original_name=ReLU)
        (conv2): RecursiveScriptModule(original_name=Conv2d)
        (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      )
      (1): RecursiveScriptModule(
        original_name=BasicBlock
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
        (relu):

In [6]:
from jetcam.usb_camera import USBCamera
camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number
camera.running = True
print("camera created")

camera created


In [7]:
import ipywidgets
import traitlets
from jetcam.utils import bgr8_to_jpeg
camera.unobserve_all()
camera_widget = ipywidgets.Image(layout = ipywidgets.Layout(width='212px'), description='cam preview:',)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)


In [8]:
from os import walk

filenames = next(walk('test_images'), (None, None, []))[2] 
# print(filenames)

In [9]:
import threading
import time
import json
import requests
import base64
from utils import preprocess
import torch.nn.functional as F

files_widget=ipywidgets.Select( options=filenames, description='image:',)
sw_widget=ipywidgets.RadioButtons(options=['camera', 'photo'], description='image source:',)
url_widget=ipywidgets.Text(description='server URL:',placeholder='http://127.0.0.1:8888/')
state_widget = ipywidgets.ToggleButtons(options=['stop', 'live','exit'], description='state', value='stop')
prediction_widget = ipywidgets.Text(description='time')

def live(state_widget, model, camera, prediction_widget):
    global dataset
    while state_widget.value == 'live':
        if sw_widget.value == 'camera':
            image = camera.value
        else:
            image = np.array(Image.open('test_images/'+files_widget.value))
        preprocessed = preprocess(image)
        # prediction_widget.value = str(preprocessed[0][0][0])
        since = time.time()
        output = model(preprocessed)
        output = output.detach().cpu().numpy()
        output=json.dumps([str(output.dtype), base64.b64encode(output).decode('latin1'), output.shape])
        for url_ in url_widget.value.split(','):
            try:
                r = requests.post(url=url_,data={'tensor': output},timeout=0.05)
            except Exception:
                pass
        time_elapsed = time.time() - since
        prediction_widget.value = str(time_elapsed)
            
def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget))
        execute_thread.start()
    elif change['new'] == 'exit': 
        import os
        os._exit(00)

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([
    sw_widget,
    files_widget,
    camera_widget,
    url_widget,
    prediction_widget,
    state_widget
])

# display(live_execution_widget)
print("live_execution_widget created")

live_execution_widget created


In [10]:

display(live_execution_widget)